# Description of the problem and solution

The problem was a classification task. We had EEG / EMG measurements of 3 classes that were unbalanced. We used different techinques to extract features that we used for the classification. For each EEG signal using the biosspy library we extracted the energy waves bands (alpha, beta, gamma, delta and theta). For each of these waves we calculated the mean, average, standard deviation, range and energy. These features were important to distinguish before Non-REM and REM stage. Another feature that we added was Sum of the power in all five energy bands. For each EMG signal we extracted the energy, the zero-crosses and the sign changes of the singal. EMG charactirizes brain muscle activity and these features are important to distinguish if the subject is asleep or not. With RandomForest we performed feature reduction (dimensionality reduction). Using all of the remained features we trained a Support Vector Machine which was fine-tuned using Cross-validation and Leave-one-subject-out (LOSO) validation. The model has 0.93 mean score in the LOSO cross-validation and 0.9397 in the public scoreboard.

# Include all the necessary packages

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
from sklearn.metrics import balanced_accuracy_score
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
!pip install biosppy
import biosppy as biosppy
import biosppy.signals.eeg as eeg
from sklearn.preprocessing import normalize
from numpy import save
from sklearn import preprocessing
from sklearn.model_selection import KFold

# Load the data from the CSV files

In [0]:
column_names_x = ['id']
for i in range(512):
  column_names_x.append('x'+str(i))

raw_dataset_x_1 = pd.read_csv('/content/train_eeg1.csv', names=column_names_x,
                     na_values = "?", comment='\t',
                     sep=",", skipinitialspace=True, skiprows=True)

dataset_x_1 = raw_dataset_x_1.copy()
dataset_x_1.tail()
dataset_x_1.pop("id")

raw_dataset_x_2 = pd.read_csv('/content/train_eeg2.csv', names=column_names_x,
                     na_values = "?", comment='\t',
                     sep=",", skipinitialspace=True, skiprows=True)

dataset_x_2 = raw_dataset_x_2.copy()
dataset_x_2.tail()
dataset_x_2.pop("id")

dataset_x_1 = np.array(dataset_x_1)
dataset_x_2 = np.array(dataset_x_2)

In [0]:
column_names_y = ['id','y']
raw_dataset_y = pd.read_csv('/content/train_labels.csv', names=column_names_y,
                    na_values = "?", comment='\t',
                    sep=",", skipinitialspace=True, skiprows=True)

dataset_y = raw_dataset_y.copy()
dataset_y.tail()
dataset_y.pop("id")

In [0]:
column_names_x = ['id']
for i in range(1, 513):
  column_names_x.append('x'+str(i))

raw_x_data_emg = pd.read_csv('/content/test_emg.csv', names=column_names_x,
                      sep=",", skipinitialspace=True, skiprows=True)
dataset_emg = raw_x_data_emg.copy()
dataset_emg.tail()
dataset_emg.pop("id")

# Extract Features

In [0]:
arr = []
features = []

ts_list=[] 
filtered_list=[] 
features_ts_list=[] 
theta_list=[] 
alpha_low_list=[] 
alpha_high_list=[] 
beta_list=[]
gamma_list=[] 
plf_pairs_list=[] 
plf_list=[]

for i in range(len(dataset_x_2)):
  print (i)

  ch1_sample = dataset_x_1[i]
  ch2_sample = dataset_x_2[i]

  joint = np.column_stack((ch1_sample, ch2_sample))

  joint = normalize(joint)

  ts, filtered, features_ts , theta , alpha_low , alpha_high , beta , gamma, plf_pairs, plf = eeg.eeg(joint, sampling_rate = 128, show=False)

  ts_list.append(ts)
  filtered_list.append(filtered)
  features_ts_list.append(features_ts)
  theta_list.append(theta)
  alpha_low_list.append(alpha_low) 
  alpha_high_list.append(alpha_high) 
  beta_list.append(beta)
  gamma_list.append(gamma)
  plf_pairs_list.append(plf_pairs)
  plf_list.append(plf)

In [0]:
#RANGE
list_range_theta=[]
for i in range(len(theta_list)):
  list_range_theta.append(np.ptp(theta_list[i]))

list_range_alpha_low=[]
for i in range(len(alpha_low_list)):
  list_range_alpha_low.append(np.ptp(alpha_low_list[i]))

list_range_alpha_high=[]
for i in range(len(alpha_high_list)):
  list_range_alpha_high.append(np.ptp(alpha_high_list[i]))

list_range_beta=[]
for i in range(len(beta_list)):
  list_range_beta.append(np.ptp(beta_list[i]))

list_range_gamma=[]
for i in range(len(gamma_list)):
  list_range_gamma.append(np.ptp(gamma_list[i]))

list_range_plf=[]
for i in range(len(plf_list)):
  list_range_plf.append(np.ptp(plf_list[i]))

In [0]:
#VARIANCE
list_var_theta=[]
for i in range(len(theta_list)):
  list_var_theta.append(np.var(theta_list[i]))

list_var_alpha_low=[]
for i in range(len(alpha_low_list)):
  list_var_alpha_low.append(np.var(alpha_low_list[i]))

list_var_alpha_high=[]
for i in range(len(alpha_high_list)):
  list_var_alpha_high.append(np.var(alpha_high_list[i]))

list_var_beta=[]
for i in range(len(beta_list)):
  list_var_beta.append(np.var(beta_list[i]))

list_var_gamma=[]
for i in range(len(gamma_list)):
  list_var_gamma.append(np.var(gamma_list[i]))

list_var_plf=[]
for i in range(len(plf_list)):
  list_var_plf.append(np.var(plf_list[i]))

In [0]:
#AVERAGE
list_ave_theta=[]
for i in range(len(theta_list)):
  list_ave_theta.append(np.average(theta_list[i]))

list_ave_alpha_low=[]
for i in range(len(alpha_low_list)):
  list_ave_alpha_low.append(np.average(alpha_low_list[i]))

list_ave_alpha_high=[]
for i in range(len(alpha_high_list)):
  list_ave_alpha_high.append(np.average(alpha_high_list[i]))

list_ave_beta=[]
for i in range(len(beta_list)):
  list_ave_beta.append(np.average(beta_list[i]))

list_ave_gamma=[]
for i in range(len(gamma_list)):
  list_ave_gamma.append(np.average(gamma_list[i]))

list_ave_plf=[]
for i in range(len(plf_list)):
  list_ave_plf.append(np.average(plf_list[i]))

In [0]:
#ENERGIES
list_total_energies=[]
for i in range(len(theta_list)):
  list_total_energies.append(np.sum(theta_list[i]** 2)+np.sum(alpha_low_list[i]** 2)+
                             np.sum(alpha_high_list[i]** 2)+np.sum(alpha_high_list[i]** 2)+
                             np.sum(gamma_list[i]** 2))

list_energy_theta=[]
for i in range(len(theta_list)):
  list_energy_theta.append(np.sum(theta_list[i]** 2))

list_energy_alpha_low=[]
for i in range(len(alpha_low_list)):
  list_energy_alpha_low.append(np.sum(alpha_low_list[i]** 2))

list_energy_alpha_high=[]
for i in range(len(alpha_high_list)):
  list_energy_alpha_high.append(np.sum(alpha_high_list[i]** 2))

list_energy_beta=[]
for i in range(len(beta_list)):
  list_energy_beta.append(np.sum(alpha_high_list[i]** 2))

list_energy_gamma=[]
for i in range(len(gamma_list)):
  list_energy_gamma.append(np.sum(gamma_list[i]** 2))

list_energy_plf=[]
for i in range(len(plf_list)):
  list_energy_plf.append(np.sum(plf_list[i]** 2))

In [0]:
#STD
list_std_theta=[]
for i in range(len(theta_list)):
  list_std_theta.append(np.std(theta_list[i]))

list_std_alpha_low=[]
for i in range(len(alpha_low_list)):
  list_std_alpha_low.append(np.std(alpha_low_list[i]))

list_std_alpha_high=[]
for i in range(len(alpha_high_list)):
  list_std_alpha_high.append(np.std(alpha_high_list[i]))

list_std_beta=[]
for i in range(len(beta_list)):
  list_std_beta.append(np.std(beta_list[i]))

list_std_gamma=[]
for i in range(len(gamma_list)):
  list_std_gamma.append(np.std(gamma_list[i]))

list_std_plf=[]
for i in range(len(plf_list)):
  list_std_plf.append(np.std(plf_list[i]))

In [0]:
#MEAN
list_mean_theta=[]
for i in range(len(theta_list)):
  list_mean_theta.append(np.mean(theta_list[i]))

list_mean_alpha_low=[]
for i in range(len(alpha_low_list)):
  list_mean_alpha_low.append(np.mean(alpha_low_list[i]))

list_mean_alpha_high=[]
for i in range(len(alpha_high_list)):
  list_mean_alpha_high.append(np.mean(alpha_high_list[i]))

list_mean_beta=[]
for i in range(len(beta_list)):
  list_mean_beta.append(np.mean(beta_list[i]))

list_mean_gamma=[]
for i in range(len(gamma_list)):
  list_mean_gamma.append(np.mean(gamma_list[i]))

list_mean_plf=[]
for i in range(len(plf_list)):
  list_mean_plf.append(np.mean(plf_list[i]))

In [0]:
#RATIOS of STD
theta_energy_list= np.divide(list_std_theta, list_total_energies) 
alpha_low_energy_list= np.divide(list_std_alpha_low, list_total_energies) 
alpha_high_energy_list= np.divide(list_std_alpha_high, list_total_energies) 
beta_energy_list = np.divide (list_std_beta, list_total_energies)
gamma_energy_list= np.divide(list_std_gamma, list_total_energies) 
plf_energy_list= np.divide(list_std_plf, list_total_energies) 

In [0]:
#RATIOS of MEAN
theta_energy_list_mean= np.divide(list_mean_theta, list_total_energies) 
alpha_low_energy_list_mean= np.divide(list_mean_alpha_low, list_total_energies) 
alpha_high_energy_list_mean= np.divide(list_mean_alpha_high, list_total_energies) 
beta_energy_list_mean = np.divide (list_mean_beta, list_total_energies)
gamma_energy_list_mean= np.divide(list_mean_gamma, list_total_energies) 
plf_energy_list_mean= np.divide(list_mean_plf, list_total_energies)

# Our Normalization Function

In [0]:
def normalize2(data):
  min_max_scaler = preprocessing.StandardScaler()
  data_scaled = min_max_scaler.fit_transform(data)

  return data_scaled

# Train

In [0]:
list_range_theta_arr = np.array(list_range_theta).reshape(-1,1)
list_range_alpha_low_arr = np.array(list_range_alpha_low).reshape(-1,1)
list_range_alpha_high_arr = np.array(list_range_alpha_high).reshape(-1,1)
list_range_beta_arr = np.array(list_range_beta).reshape(-1,1)
list_range_gamma_arr = np.array(list_range_gamma).reshape(-1,1)
list_range_plf_arr = np.array(list_range_plf).reshape(-1,1)
list_var_theta_arr = np.array(list_var_theta).reshape(-1,1)
list_var_alpha_low_arr = np.array(list_var_alpha_low).reshape(-1,1)
list_var_alpha_high_arr = np.array(list_var_alpha_high).reshape(-1,1)
list_var_beta_arr = np.array(list_var_beta).reshape(-1,1)
list_var_gamma_arr = np.array(list_var_gamma).reshape(-1,1)
list_var_plf_arr = np.array(list_var_plf).reshape(-1,1)
list_ave_theta_arr = np.array(list_ave_theta).reshape(-1,1)
list_ave_alpha_low_arr = np.array(list_ave_alpha_low).reshape(-1,1)
list_ave_alpha_high_arr = np.array(list_ave_alpha_high).reshape(-1,1)
list_ave_beta_arr = np.array(list_ave_beta).reshape(-1,1)
list_ave_gamma_arr = np.array(list_ave_gamma).reshape(-1,1)
list_ave_plf_arr = np.array(list_ave_plf).reshape(-1,1)
list_energy_theta_arr = np.array(list_energy_theta).reshape(-1,1)
list_energy_alpha_low_arr = np.array(list_energy_alpha_low).reshape(-1,1)
list_energy_alpha_high_arr = np.array(list_energy_alpha_high).reshape(-1,1)
list_energy_beta_arr = np.array(list_energy_beta).reshape(-1,1)
list_energy_gamma_arr = np.array(list_energy_gamma).reshape(-1,1)
list_energy_plf_arr = np.array(list_energy_plf).reshape(-1,1)
list_total_energies= np.array(list_total_energies).reshape(-1,1)
list_std_theta_arr = np.array(list_std_theta).reshape(-1,1)
list_std_alpha_low_arr = np.array(list_std_alpha_low).reshape(-1,1)
list_std_alpha_high_arr = np.array(list_std_alpha_high).reshape(-1,1)
list_std_beta_arr = np.array(list_std_beta).reshape(-1,1)
list_std_gamma_arr = np.array(list_std_gamma).reshape(-1,1)
list_std_plf_arr = np.array(list_std_plf).reshape(-1,1)
list_mean_theta_arr = np.array(list_mean_theta).reshape(-1,1)
list_mean_alpha_low_arr = np.array(list_mean_alpha_low).reshape(-1,1)
list_mean_alpha_high_arr = np.array(list_mean_alpha_high).reshape(-1,1)
list_mean_beta_arr = np.array(list_mean_beta).reshape(-1,1)
list_mean_gamma_arr = np.array(list_mean_gamma).reshape(-1,1)
list_mean_plf_arr = np.array(list_mean_plf).reshape(-1,1)
theta_energy_list= np.array(theta_energy_list).reshape(-1,1)
alpha_low_energy_list= np.array(alpha_low_energy_list).reshape(-1,1)
alpha_high_energy_list= np.array(alpha_high_energy_list).reshape(-1,1)
beta_energy_list= np.array(beta_energy_list).reshape(-1,1)
gamma_energy_list= np.array(gamma_energy_list).reshape(-1,1)
theta_energy_list_mean= np.array(theta_energy_list_mean).reshape(-1,1)
alpha_low_energy_list_mean= np.array(alpha_low_energy_list_mean).reshape(-1,1)
alpha_high_energy_list_mean= np.array(alpha_high_energy_list_mean).reshape(-1,1) 
beta_energy_list_mean= np.array(beta_energy_list_mean).reshape(-1,1)
gamma_energy_list_mean= np.array(gamma_energy_list_mean).reshape(-1,1)


features_arr=np.concatenate((list_range_delta_arr,
                             list_range_theta_arr,
                             list_range_alpha_low_arr,
                             list_range_alpha_high_arr,
                             list_range_beta_arr,
                             list_range_gamma_arr,
                             list_ave_delta_arr,
                             list_ave_theta_arr,
                             list_ave_alpha_low_arr,
                             list_ave_alpha_high_arr,
                             list_ave_beta_arr,
                             list_ave_gamma_arr,
                             list_std_delta_arr,
                             list_std_theta_arr,
                             list_std_alpha_low_arr,
                             list_std_alpha_high_arr,
                             list_std_beta_arr,
                             list_std_gamma_arr,
                             list_energy_delta_arr,
                             list_energy_theta_arr,
                             list_energy_alpha_low_arr,
                             list_energy_alpha_high_arr,
                             list_energy_beta_arr,
                             list_energy_gamma_arr,
                             list_total_energies,
                             delta_energy_list,
                             theta_energy_list,
                             alpha_low_energy_list,
                             alpha_high_energy_list, 
                             beta_energy_list,
                             gamma_energy_list,
                             list_mean_delta_arr,
                             list_mean_theta_arr,
                             list_mean_alpha_low_arr,
                             list_mean_alpha_high_arr,
                             list_mean_beta_arr,
                             list_mean_gamma_arr
                             ),axis=1)
features_arr.shape

(64800, 29)

# Load previous Medium

In [0]:
eeg_avg = np.load("eeg_avgs_train_scaled.npy")
eeg_varptp = np.load("eeg_var_ptp_train_scaled.npy")

emg3 = np.load("/content/emg_features_3.npy")
emg3 = normalize2(emg3)

features_arr_old = np.column_stack((eeg_avg,eeg_varptp,emg3))

features_arr_old = np.delete(features_arr_old,[5,17,11],axis=1)
features_arr_old.shape

(64800, 27)

# Combine

In [0]:
features_arr=np.concatenate((features_arr,features_arr_old), axis=1)
features_arr.shape

(64800, 56)

# Cross Val

In [0]:
scores, members = list(), list()

x_data = features_arr # !!!!!!!!!!!!!!!!!!!!! prep_trainData()
y = np.array(dataset_y)

repeat = 1 # number of times to do the k-fold cross validation
n_folds = 10 # of folds for the cross validation

models_list = []

for i in range(repeat):

  kfold = KFold(n_folds, True, i) # change seed each time
  
# cross validation
  for train_ix, test_ix in kfold.split(x_data, y):

    print("::::::::   # of Training Indices: ", train_ix.size,
          "::::::::::   # of Testing Indices: ", test_ix.size)

    # select samples
    x_train = []
    y_train = []
    x_test = []
    y_test = []

    for i in range(train_ix.size):
      x_train.append(x_data[train_ix[i]])
      y_train.append(y[train_ix[i]])

    for i in range(test_ix.size):
      x_test.append(x_data[test_ix[i]])
      y_test.append(y[test_ix[i]])

    y_train = np.array(y_train) 
    x_train = np.array(x_train)
    y_test = np.array(y_test) 
    x_tests= np.array(x_test) 

    print ('Training')
    clf = SVC(class_weight='balanced', kernel='rbf', gamma='scale', random_state=0)
    clf.fit(x_train, y_train)
    models_list.append(clf)
    print ('Predicting')
    predicted_labels = clf.predict(x_test)
    print ('Scoring')     
    score = balanced_accuracy_score(y_test, predicted_labels)
    print('>%.3f' % score)
    scores.append(score)

In [0]:
print("avg: ", np.average(scores), "\nvar: ", np.var(scores))
plt.plot(scores)  

# Leave one out Validation

In [0]:
x_data = features_arr
y = np.array(dataset_y)

labels1 = y[0:21600]
labels2 = y[21600:43200]
labels3 = y[43200:y.shape[0]]

patient1 = x_data[0:21600]
patient2 = x_data[21600:43200]
patient3 = x_data[43200:x_data.shape[0]]

clf = SVC(class_weight='balanced',kernel='rbf',gamma='scale', random_state=0, decision_function_shape='ovo')

clf.fit(np.concatenate((patient1, patient2)), np.concatenate((labels1, labels2)))
y_pred = clf.predict(patient3)
bmac3 = balanced_accuracy_score(labels3, y_pred)

clf.fit(np.concatenate((patient1, patient3)), np.concatenate((labels1, labels3)))
y_pred = clf.predict(patient2)
bmac2 = balanced_accuracy_score(labels2, y_pred)

clf.fit(np.concatenate((patient2, patient3)), np.concatenate((labels2, labels3)))
y_pred = clf.predict(patient1)
bmac1 = balanced_accuracy_score(labels1, y_pred)

print("Test on Patient1: ", bmac1) 
print("Test on Patient2: ", bmac2) 
print("Test on Patient3: ", bmac3) 
print ("Average Validation Score: ", (bmac1+bmac2+bmac3)/3)

# Train whole dataset

In [0]:
x_data = features_arr
y = np.array(dataset_y)
clf = SVC(class_weight='balanced',kernel='rbf',gamma='scale', random_state=0)
clf.fit(x_data,y)

# Load my test features

In [0]:
list_range_theta_arr = np.array(list_range_theta).reshape(-1,1)
list_range_alpha_low_arr = np.array(list_range_alpha_low).reshape(-1,1)
list_range_alpha_high_arr = np.array(list_range_alpha_high).reshape(-1,1)
list_range_beta_arr = np.array(list_range_beta).reshape(-1,1)
list_range_gamma_arr = np.array(list_range_gamma).reshape(-1,1)
list_range_plf_arr = np.array(list_range_plf).reshape(-1,1)
list_var_theta_arr = np.array(list_var_theta).reshape(-1,1)
list_var_alpha_low_arr = np.array(list_var_alpha_low).reshape(-1,1)
list_var_alpha_high_arr = np.array(list_var_alpha_high).reshape(-1,1)
list_var_beta_arr = np.array(list_var_beta).reshape(-1,1)
list_var_gamma_arr = np.array(list_var_gamma).reshape(-1,1)
list_var_plf_arr = np.array(list_var_plf).reshape(-1,1)
list_ave_theta_arr = np.array(list_ave_theta).reshape(-1,1)
list_ave_alpha_low_arr = np.array(list_ave_alpha_low).reshape(-1,1)
list_ave_alpha_high_arr = np.array(list_ave_alpha_high).reshape(-1,1)
list_ave_beta_arr = np.array(list_ave_beta).reshape(-1,1)
list_ave_gamma_arr = np.array(list_ave_gamma).reshape(-1,1)
list_ave_plf_arr = np.array(list_ave_plf).reshape(-1,1)
list_energy_theta_arr = np.array(list_energy_theta).reshape(-1,1)
list_energy_alpha_low_arr = np.array(list_energy_alpha_low).reshape(-1,1)
list_energy_alpha_high_arr = np.array(list_energy_alpha_high).reshape(-1,1)
list_energy_beta_arr = np.array(list_energy_beta).reshape(-1,1)
list_energy_gamma_arr = np.array(list_energy_gamma).reshape(-1,1)
list_energy_plf_arr = np.array(list_energy_plf).reshape(-1,1)
list_total_energies= np.array(list_total_energies).reshape(-1,1)
list_std_theta_arr = np.array(list_std_theta).reshape(-1,1)
list_std_alpha_low_arr = np.array(list_std_alpha_low).reshape(-1,1)
list_std_alpha_high_arr = np.array(list_std_alpha_high).reshape(-1,1)
list_std_beta_arr = np.array(list_std_beta).reshape(-1,1)
list_std_gamma_arr = np.array(list_std_gamma).reshape(-1,1)
list_std_plf_arr = np.array(list_std_plf).reshape(-1,1)
list_mean_theta_arr = np.array(list_mean_theta).reshape(-1,1)
list_mean_alpha_low_arr = np.array(list_mean_alpha_low).reshape(-1,1)
list_mean_alpha_high_arr = np.array(list_mean_alpha_high).reshape(-1,1)
list_mean_beta_arr = np.array(list_mean_beta).reshape(-1,1)
list_mean_gamma_arr = np.array(list_mean_gamma).reshape(-1,1)
list_mean_plf_arr = np.array(list_mean_plf).reshape(-1,1)
theta_energy_list= np.array(theta_energy_list).reshape(-1,1)
alpha_low_energy_list= np.array(alpha_low_energy_list).reshape(-1,1)
alpha_high_energy_list= np.array(alpha_high_energy_list).reshape(-1,1)
beta_energy_list= np.array(beta_energy_list).reshape(-1,1)
gamma_energy_list= np.array(gamma_energy_list).reshape(-1,1)
theta_energy_list_mean= np.array(theta_energy_list_mean).reshape(-1,1)
alpha_low_energy_list_mean= np.array(alpha_low_energy_list_mean).reshape(-1,1)
alpha_high_energy_list_mean= np.array(alpha_high_energy_list_mean).reshape(-1,1) 
beta_energy_list_mean= np.array(beta_energy_list_mean).reshape(-1,1)
gamma_energy_list_mean= np.array(gamma_energy_list_mean).reshape(-1,1)


features_arr_test=np.concatenate((list_range_delta_arr,
                             list_range_theta_arr,
                             list_range_alpha_low_arr,
                             list_range_alpha_high_arr,
                             list_range_beta_arr,
                             list_range_gamma_arr,
                             list_ave_delta_arr,
                             list_ave_theta_arr,
                             list_ave_alpha_low_arr,
                             list_ave_alpha_high_arr,
                             list_ave_beta_arr,
                             list_ave_gamma_arr,
                             list_std_delta_arr,
                             list_std_theta_arr,
                             list_std_alpha_low_arr,
                             list_std_alpha_high_arr,
                             list_std_beta_arr,
                             list_std_gamma_arr,
                             list_energy_delta_arr,
                             list_energy_theta_arr,
                             list_energy_alpha_low_arr,
                             list_energy_alpha_high_arr,
                             list_energy_beta_arr,
                             list_energy_gamma_arr,
                             list_total_energies,
                             delta_energy_list,
                             theta_energy_list,
                             alpha_low_energy_list,
                             alpha_high_energy_list, 
                             beta_energy_list,
                             gamma_energy_list,
                             list_mean_delta_arr,
                             list_mean_theta_arr,
                             list_mean_alpha_low_arr,
                             list_mean_alpha_high_arr,
                             list_mean_beta_arr,
                             list_mean_gamma_arr
                             ),axis=1)
features_arr_test.shape

(43200, 29)

# Load previous Medium test

In [0]:
eeg_avg = np.load("eeg_avg_test_scaled.npy")
eeg_varptp = np.load("eeg_var_ptp_test.npy")
eeg_varptp = normalize2(eeg_varptp)

emg3 = np.load("/content/emg_features_3_test.npy")
emg3 = normalize2(emg3)

features_arr_old = np.column_stack((eeg_avg,eeg_varptp,emg3))

features_arr_old = np.delete(features_arr_old,[5,17,11],axis=1)
features_arr_old.shape

(43200, 24)

# Combine for Test

In [0]:
features_arr_test=np.concatenate((features_arr_test,features_arr_old), axis=1)
features_arr_test.shape

(43200, 53)

# Extract

In [0]:
y_pred = clf.predict(features_arr_test)

index = 0.0
with open('final.txt', 'w') as f:
    f.write("%s\n" % "Id,y")
    for predict in y_pred:
        writing_str = str(index)+','+str(predict.item(0))
        f.write("%s\n" % writing_str)
        index = index + 1